In [1]:
cd E:/GitHub/computervision

e:\GitHub\computervision


In [2]:
import torch
import os

import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from torch.optim import lr_scheduler
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import ViT

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'
MODEL_WEIGHTS_PATH = None
if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
DATA_DIR = ['E:/Datasets/cor-splits/sgkf-8-1-1-size-540', 'E:/Datasets/cor-splits/sgkf-8-1-1-size-1010']
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 64
SAVE_INTERVAL = 2 # How many epochs between each backup
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of spochs before starting patience counter

In [4]:
vit = ViT()

model = vit.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()

if FULLY_TRAIN:
    params = model.parameters()
else:
    params = model.heads.parameters()
    
# optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
optimizer = optim.SGD(params, lr=LR)
# optimizer = optim.Adamax(model.parameters(), lr=LR)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)

In [8]:
for dir in DATA_DIR:
    dataset = PytorchDataset(dir, vit.data_transforms, BATCH_SIZE)

    working_folder = 'notebooks/vit'
    models_folder = 'models'

    curr_time = datetime.now()
    curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

    output_folder = os.path.join(working_folder, models_folder)
    output_folder = os.path.join(output_folder, curr_time)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    trainer = PytorchTraining(device, dataset, output_folder)
    model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/50
----------
train Loss: 0.7650 Acc: 0.4977
val Loss: 0.8036 Acc: 0.3704
Epoch duration: 0 m 45s
Learning Rate = 0.001


Epoch 2/50
----------
train Loss: 0.7034 Acc: 0.5278
val Loss: 0.7786 Acc: 0.4074
Epoch duration: 0 m 37s
Learning Rate = 0.001


Epoch 3/50
----------
train Loss: 0.6623 Acc: 0.5995
val Loss: 0.7525 Acc: 0.4444
Epoch duration: 1 m 1s
Learning Rate = 0.001


Epoch 4/50
----------
train Loss: 0.6364 Acc: 0.6435
val Loss: 0.7171 Acc: 0.4815
Epoch duration: 1 m 19s
Learning Rate = 0.001


Epoch 5/50
----------
train Loss: 0.6059 Acc: 0.7176
val Loss: 0.7042 Acc: 0.5000
Epoch duration: 1 m 21s
Learning Rate = 0.001


Epoch 6/50
----------
train Loss: 0.5814 Acc: 0.7361
val Loss: 0.6867 Acc: 0.5370
Epoch duration: 1 m 14s
Learning Rate = 0.001


Epoch 7/50
----------
train Loss: 0.5566 Acc: 0.7523
val Loss: 0.6702 Acc: 0.5370
Epoch duration: 0 m 55s
Learning Rate = 0.001


Epoch 8/50
----------
train Loss: 0.5327 Acc: 0.7940
val Loss: 0.6549 Acc: 0.5741
Epoch dur